白ヤギのモデルを利用

In [58]:
import pandas as pd
import numpy as np

import MeCab
import gensim
from gensim.models import word2vec

import logging
import sys

In [59]:
#snssns..set_styleset_sty ('whitegrid')
%matplotlib inline

pd.set_option("display.max_columns", 200) # 表示カラムの最大値
pd.set_option('display.max_rows', 200) # 表示行数の最大値
pd.set_option('display.max_colwidth', 30)
pd.set_option("display.float_format", '{:.3f}'.format) # 小数点以下の有効数字

# jupyterの幅を自動調整
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:85% !important; }</style>"))

# モデル読み込み

参照：　http://aial.shiroyagi.co.jp/2017/02/japanese-word2vec-model-builder/

In [7]:
model_path = '02.model/word2vec.gensim.model'
model = Word2Vec.load(model_path)

# 単語／文章の類似度

## 単語の類似度を算出

In [60]:
# 類似度の高い単語を表示
for r in model.wv.most_similar(positive=['上杉謙信']):
    print(r)

('武田信玄', 0.9536293745040894)
('毛利元就', 0.9534527659416199)
('今川義元', 0.9445849657058716)
('織田信長', 0.9392962455749512)
('長宗我部元親', 0.9386113882064819)
('大友宗麟', 0.9382531046867371)
('最上義光', 0.9372270107269287)
('里見氏', 0.9340117573738098)
('北条氏康', 0.9319975972175598)
('大友氏', 0.928727388381958)


## 文章の類似度を算出

In [67]:
mt = MeCab.Tagger('')
mt.parse('')

# テキストのベクトルを計算
def get_vector(text):
    sum_vec = np.zeros(model.vector_size)
    word_count = 0
    node = mt.parseToNode(text)
    
    while node:
        fields = node.feature.split(",")
        # 名詞、動詞、形容詞に限定
        if fields[0] == '名詞' or fields[0] == '動詞' or fields[0] == '形容詞':
            sum_vec += model.wv[node.surface]
            word_count += 1
        node = node.next

    return sum_vec / word_count

# cos類似度を計算
def cos_sim(v1, v2):
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))


In [71]:
if __name__ == "__main__":
    v1 = get_vector('国内の物流大手がEVを本格導入するのは初めて。')
    v2 = get_vector('徳川家康が江戸幕府をひらいた')
    v3 = get_vector('国際物流大手の独ドイツポストDHLグループと、宅配便の配送に使う小型の電気自動車（EV）を共同開発する。')
    
    print(cos_sim(v1, v2))
    print(cos_sim(v1, v3))
    print(cos_sim(v2, v3))

-0.060320016599344585
0.7987708061912694
-0.09940524226880823


## 単語の増減を計算して類似度を算出

In [72]:
model.wv.most_similar(positive=['王様','女性'],negative=['男性'])

[('お姫様', 0.8531286716461182),
 ('花嫁', 0.839175820350647),
 ('野獣', 0.8315544128417969),
 ('魔女', 0.8298223614692688),
 ('乙女', 0.8235582113265991),
 ('愛する人', 0.8003491163253784),
 ('天国', 0.7905834913253784),
 ('悪魔', 0.7898155450820923),
 ('キューピッド', 0.7896509170532227),
 ('魔法使い', 0.788610577583313)]

# トピックモデルの検証